In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LayerNormalization
from tensorflow.keras.optimizers import RMSprop
from sklearn.model_selection import KFold
from sklearn import preprocessing, metrics

In [1]:
slug = pd.read_csv("220306_slugs_dataset.csv")

NameError: name 'pd' is not defined

In [3]:
slug.head()

,loadCase,slugDen,slugLen,pockDen,pockLen,slugVel,basePer,baseAmp,basePha,simTime,out_oopDisp_mm_n2,out_supReac_kN_n3,out_supReac_kN_n4,out_verDisp_mm_n5,out_oopDisp_mm_n5,out_endBend_kNm_n6
0,00001_sd786_sl6_pd234_pl10_sv6_bp13_ba850_bp74,785.67,6.47,234.16,10.06,5.77,12.55,0.85,74.10,45.8,197.45,16.22,56.42,248.24,180.47,198.49
1,00002_sd705_sl10_pd190_pl11_sv7_bp12_ba800_bp59,705.03,10.36,190.44,11.25,7.42,11.63,0.80,59.11,44.0,186.85,15.97,58.86,262.33,178.72,238.07
2,00003_sd742_sl12_pd159_pl13_sv7_bp12_ba650_bp65,741.98,11.64,159.09,12.52,7.42,11.54,0.65,65.23,48.2,155.47,14.29,48.63,220.89,163.53,180.64
3,00004_sd732_sl9_pd168_pl10_sv10_bp11_ba670_bp68,731.88,8.82,167.88,9.84,9.52,11.48,0.67,67.85,30.8,169.47,15.18,48.72,188.47,163.76,233.31
4,00005_sd669_sl11_pd214_pl9_sv8_bp13_ba810_bp57,669.21,11.46,213.87,8.56,8.18,12.86,0.81,56.68,37.7,223.64,16.90,58.51,271.74,242.37,219.33


In [4]:
slug = slug.dropna()

In [5]:
#Shuffle the dataset
slug = slug.sample(frac=1).reset_index(drop=True)

In [6]:
#Select columns
slug_df = slug[["slugDen","slugLen","pockDen","pockLen","slugVel","basePer", "baseAmp", "basePha"]]

#Create new features
slug_df['densDiff'] = slug_df['slugDen'] - slug_df['pockDen']
slug_df['centrFor'] = slug_df['slugVel']**2 * slug_df['densDiff']

C:\Users\mania\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\mania\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
scaler = preprocessing.StandardScaler()
scaler = scaler.fit(slug_df)

In [8]:
#Create Targets
slug_tar = slug[["out_oopDisp_mm_n2", "out_oopDisp_mm_n5", "out_endBend_kNm_n6"]]

In [9]:
slug_tar.head()

,out_oopDisp_mm_n2,out_oopDisp_mm_n5,out_endBend_kNm_n6
0,165.72,173.47,217.89
1,183.48,209.82,304.90
2,171.02,155.02,148.29
3,215.69,229.76,276.82
4,197.09,195.00,241.16


In [10]:
#Scale independent variables
slug_train_df = np.array(scaler.transform(slug_df))

In [11]:
#First 40,000 used to train model, test on remainder
train_df = np.array(scaler.transform(slug_df[:40000]))
train_tar = np.array(slug_tar[:40000])
test_df = np.array(scaler.transform(slug_df[40000:]))
test_tar = np.array(slug_tar[40000:])

In [12]:
def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(256, input_dim=n_inputs, activation='relu'))
    model.add(Dropout(.4))
    model.add(Dense(512, activation='relu'))
    
    #model.add(Dense(1024, activation='relu'))
    
    model.add(Dense(1024, activation='relu'))

    model.add(Dense(512, activation='relu'))
    
    model.add(Dense(n_outputs, activation='linear'))
    model.compile(loss='mean_squared_error', optimizer=RMSprop(lr=0.01, clipnorm=1.), metrics=['mean_squared_error'])
    #model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])
    return model

In [13]:
n_inputs, n_outputs = train_df.shape[1], train_tar.shape[1]

In [14]:
model = get_model(n_inputs, n_outputs)

In [15]:
model.fit(slug_train_df, slug_tar.values, epochs=20, batch_size=20)

Train on 49975 samples
Epoch 1/20
49975/49975 [==============================] - 8s 151us/sample - loss: 1941.6779 - mean_squared_error: 1941.6733
Epoch 2/20
49975/49975 [==============================] - 7s 141us/sample - loss: 747.7297 - mean_squared_error: 747.7295
Epoch 3/20
49975/49975 [==============================] - 7s 142us/sample - loss: 645.1292 - mean_squared_error: 645.1297
Epoch 4/20
49975/49975 [==============================] - 7s 142us/sample - loss: 600.7610 - mean_squared_error: 600.7610
Epoch 5/20
49975/49975 [==============================] - 7s 142us/sample - loss: 578.6922 - mean_squared_error: 578.6924
Epoch 6/20
49975/49975 [==============================] - 7s 142us/sample - loss: 544.2664 - mean_squared_error: 544.2667
Epoch 7/20
49975/49975 [==============================] - 7s 142us/sample - loss: 539.5938 - mean_squared_error: 539.5941
Epoch 8/20
49975/49975 [==============================] - 7s 142us/sample - loss: 516.3435 - mean_squared_error: 516.3438

In [16]:
#Use model to make predictions for test set
preds = model.predict(test_df)

In [17]:
preds

array([[189.7469 , 199.37271, 208.18307],
       [154.96716, 147.26416, 171.42924],
       [259.12854, 277.01154, 323.70425],
       ...,
       [186.21307, 179.67941, 252.92368],
       [202.6653 , 190.1566 , 315.26868],
       [180.29456, 184.0963 , 271.7378 ]], dtype=float32)

In [18]:
test_tar

array([[200.62, 231.98, 185.62],
       [146.85, 130.01, 181.98],
       [284.99, 335.37, 332.78],
       ...,
       [180.33, 177.57, 238.21],
       [206.89, 198.41, 322.11],
       [174.95, 184.36, 232.7 ]])

In [19]:
mae = metrics.mean_absolute_error(test_tar, preds)
mse = metrics.mean_squared_error(test_tar, preds)
rmse = np.sqrt(mse) # or mse**(0.5)  
r2 = metrics.r2_score(test_tar, preds)

In [20]:
print("MAE:", round(mae, 2))
print("MSE:", round(mse, 2))
print("RMSE:", round(rmse, 2))
print("R-Squared:", round(r2, 2))

MAE: 13.46
MSE: 367.51
RMSE: 19.17
R-Squared: 0.83


In [ ]:
#Without new features
MAE: 15.33
MSE: 502.25
RMSE: 22.41
R-Squared: 0.73